In [ ]:
!pip install scipy==1.1.0

     |████████████████████████████████| 31.2 MB 65 kB/s 
  Attempting uninstall: scipy
    Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pymc3 3.11.4 requires scipy>=1.2.0, but you have scipy 1.1.0 which is incompatible.
plotnine 0.6.0 requires scipy>=1.2.0, but you have scipy 1.1.0 which is incompatible.
jax 0.2.21 requires scipy>=1.2.1, but you have scipy 1.1.0 which is incompatible.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import skimage 
from skimage import data,io
import os 
import shutil
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Flatten, Dense, Input, TimeDistributed
import tensorflow as tf
from tensorflow.keras.applications.resnet_v2 import ResNet50V2,preprocess_input,decode_predictions
from keras import backend as K
from keras.preprocessing.image import load_img, img_to_array
from collections import Counter, defaultdict
from PIL import ImageDraw
from scipy.misc import imread, imresize, imsave, fromimage, toimage
try:
    from io import BytesIO
except ImportError:
    from StringIO import StringIO as BytesIO
import PIL
from IPython.display import clear_output, Image, display, HTML
from keras.callbacks import ModelCheckpoint,ReduceLROnPlateau,EarlyStopping

In [ ]:
img_size=(224,224)
batch_size=32

In [ ]:
train_ds=tf.keras.preprocessing.image_dataset_from_directory(
    '/content/drive/MyDrive/AI/S4/pj4/train',
    labels="inferred",
    label_mode="int",
    class_names=None,
    color_mode="rgb",
    batch_size=batch_size,
    image_size=img_size,
    shuffle=True,
    seed=123,
    validation_split=0.2,
    subset='training',
    interpolation="bilinear",
    follow_links=False,
    crop_to_aspect_ratio=False,    
)

Found 8916 files belonging to 2 classes.
Using 7133 files for training.


In [ ]:
vali_ds=tf.keras.preprocessing.image_dataset_from_directory(
    '/content/drive/MyDrive/AI/S4/pj4/train',
    labels="inferred",
    label_mode="int",
    class_names=None,
    color_mode="rgb",
    batch_size=batch_size,
    image_size=img_size,
    shuffle=True,
    seed=123,
    validation_split=0.2,
    subset='validation',
    interpolation="bilinear",
    follow_links=False,
    crop_to_aspect_ratio=False,    
)

Found 8916 files belonging to 2 classes.
Using 1783 files for validation.


In [ ]:
test_ds=tf.keras.preprocessing.image_dataset_from_directory(
    '/content/drive/MyDrive/AI/S4/pj4/test',
    labels="inferred",
    label_mode="int",
    class_names=None,
    color_mode="rgb",
    batch_size=batch_size,
    image_size=img_size,
    shuffle=True,
    seed=123,    
    interpolation="bilinear",
    follow_links=False,
    crop_to_aspect_ratio=False,    
)

Found 2228 files belonging to 2 classes.


In [ ]:
train=train_ds
vali=vali_ds
test=test_ds

In [ ]:
class_names = train.class_names
print(class_names)

['insta', 'porn']


In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

train= train.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
vali =vali.cache().prefetch(buffer_size=AUTOTUNE)
test =test.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
base_model = ResNet50V2(weights='imagenet', include_top=True)
base_model.trainable = True

102883328/102869336 [==============================] - 1s 0us/step


In [ ]:
 import matplotlib.pyplot as plt

In [ ]:
# class DisplayCallback(tf.keras.callbacks.Callback):
#   def on_epoch_end(self, epoch, logs=None):
#     clear_output(wait=True)
#     show_predictions()
#     print ('\n에포크 이후 예측 예시 {}\n'.format(epoch+1))

In [ ]:
preprocess_input = tf.keras.layers.experimental.preprocessing.Rescaling(scale=1./255)
IMG_SHAPE = img_size + (3,)
#prediction_layer = tf.keras.layers.Dense(len(class_names)) 다중이면 출력층에 이걸써도될듯

In [ ]:
def show_predictions(dataset=None, num=1): 
     
    for image, lable in dataset.take(num):
      plt.figure(figsize=(10, 10))      
      pred = model.predict(image)
      plt.axis('off')     
      plt.imshow(image/255)
      
      print('labe : ',lable,'pred : ',pred) 

In [ ]:
model_ckpt = ModelCheckpoint('/content/drive/MyDrive/AI/S4/pj4/model_ckpt.h5',save_best_only=True)
early_stop = EarlyStopping(patience=5,verbose=2,monitor='loss')
reduce_lr = ReduceLROnPlateau(patience=4,verbose=1)

In [ ]:
# #모델 처음부터 학습할때만 킬것
# inputs = tf.keras.Input(shape=IMG_SHAPE)
# x = preprocess_input(inputs)
# x = base_model(x,training=True)
# x = tf.keras.layers.Dropout(0.2)(x)
# outputs = tf.keras.layers.Dense(1,activation='sigmoid')(x)
# model = tf.keras.Model(inputs, outputs)


In [ ]:
#저장했던 모델을 불러와서 이어서 학습
model = tf.keras.models.load_model('/content/drive/MyDrive/AI/S4/pj4/model_ckpt.h5')


In [ ]:
model.compile(optimizer='adam',
              loss='BinaryCrossentropy',#tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 rescaling (Rescaling)       (None, 224, 224, 3)       0         
                                                                 
 resnet50v2 (Functional)     (None, 1000)              25613800  
                                                                 
 dropout (Dropout)           (None, 1000)              0         
                                                                 
 dense (Dense)               (None, 1)                 1001      
                                                                 
Total params: 25,614,801
Trainable params: 25,569,361
Non-trainable params: 45,440
_________________________________________________________________


In [ ]:
# EPOCHS = 100

# #모델학습
# model_history = model.fit(train, epochs=EPOCHS,
#                           validation_data=vali,
#                           callbacks=[model_ckpt,reduce_lr,early_stop])

In [ ]:
# acc = model_history.history['accuracy']
# val_acc = model_history.history['val_accuracy']

# loss = model_history.history['loss']
# val_loss = model_history.history['val_loss']

# plt.figure(figsize=(8, 8))
# plt.subplot(2, 1, 1)
# plt.plot(acc, label='Training Accuracy')
# plt.plot(val_acc, label='Validation Accuracy')
# plt.legend(loc='lower right')
# plt.ylabel('Accuracy')
# plt.ylim([min(plt.ylim()),1])
# plt.title('Training and Validation Accuracy')

# plt.subplot(2, 1, 2)
# plt.plot(loss, label='Training Loss')
# plt.plot(val_loss, label='Validation Loss')
# plt.legend(loc='upper right')
# plt.ylabel('Cross Entropy')
# plt.ylim([0,1.0])
# plt.title('Training and Validation Loss')
# plt.xlabel('epoch')
# plt.show()
#학습할떄만 킬것

In [ ]:
test_loss, test_acc = model.evaluate(test, verbose=2)

70/70 - 332s - loss: 0.0332 - accuracy: 0.9937 - 332s/epoch - 5s/step


In [ ]:
# model.save('/content/drive/MyDrive/AI/S4/pj4/model_ckpt.h5') 
#모델 수동세이브 학습할때만 킬것


In [ ]:
def show_predictions(dataset=None, num=1): 
     
    for image, lable in dataset.take(num):      
        plt.figure(figsize=(10, 10))      
        pred = model.predict(image)
        pred_1=(pred > 0.5).astype("int32")
        plt.axis('off')    
        #plt.xlabel('labe : ',lable[num],'pred : ',pred[num]) 
        plt.imshow(image[num]/255)        
        plt.text(1.0, -1, f'number : {num} , lable : {lable[num]} ,  pred : {pred[num]}, class :{pred_1[num]}')        

def preprocess_image(image_path, target_size=None):
    img = load_img(image_path, target_size=target_size)
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)    
    return img

def isokay(pic):
    pol=0        
    img=preprocess_image(pic, target_size=img_size)
    plt.figure(figsize=(8, 8))              
    plt.axis('off')    
    pred=model.predict(img)[0][0]
    if pred >=0.8 :
        pol=1
        say= "{:.3f}% 확률로 음란물은 너굴맨이 처리했으니 안심하라구".format(pred*100)    
    else :
      say= "{:.3f}% 확률로 음란물 ".format(pred*100)   
    if pol ==1:
      plt.imshow(load_img('/content/drive/MyDrive/AI/S4/pj4/너굴.jpg'))
    else:
      plt.imshow(load_img(pic))
    print('Predicted:', say)    
            
      

In [ ]:
#show_predictions(test, 1)

In [ ]:
# from google.colab import files
# files.upload()

In [ ]:
#isokay('/content/drive/MyDrive/AI/S4/pj4/img/0.png')